In [1]:
import cv2 as cv
import numpy as np
import time
import tensorflow as tf

import matplotlib.pyplot as plt
%matplotlib inline

# Model

In [2]:
# Load Pretrained Model
path_model = r'C:\Users\Fernanda Januar\Documents\Adobe\Premiere Pro\14.0\Fire_Classification_mobilenetv2.h5'
model = tf.keras.models.load_model(path_model)

In [7]:
'''path = r'C:\Users\Fernanda Januar\Downloads\fire4.mp4'
vid = cv.VideoCapture(path)

f = 0
t = 0
while True:
    ret, frame = vid.read()
    if not ret:
        break
    
    # Resize the Frame to 224,224,3 and convert to 1,224,224,3-----------
    img_resize = tf.keras.preprocessing.image.smart_resize(
        frame,
        (224, 224)
    )
    new_frame = np.expand_dims(img_resize, 0)
    
    # Predict the frames/images
    start = time.time()
    label_pred = np.argmax(model.predict(new_frame))
    
    end = time.time()
    f+=1
    t+=end-start
    #--------------------------------------------------------------------
    
    #Labels---------------------------------------------------------------------------------
    if label_pred==1:
        color = [0,0,255]
        lol = "FIRE"
    else:
        color = [255,0,0]
        lol = "Not_FIRE"
    
    text1 = "Predict :: {}".format(lol)
    text2 = "Frame number {0} took {1:.5f} seconds".format(f, end - start)
    text3 = "FPS: {}".format(round((f / t), 1))
    cv.putText(frame, text1, (30, 150), cv.FONT_HERSHEY_COMPLEX, 1.0, color, 2)
    cv.putText(frame, text2, (30, 190), cv.FONT_HERSHEY_COMPLEX, 1.0, (255,0,0), 2)
    cv.putText(frame, text3, (30, 230), cv.FONT_HERSHEY_COMPLEX, 1.0, (255,0,0), 2)
    #---------------------------------------------------------------------------------------
    
    #sizesize = cv.resize(frame, (224,224))
    cv.imshow("Frame", frame)
    if cv.waitKey(1) & 0xFF==ord("q"):
        break
    
    #plt.imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB), cmap='gray')
    #print(frame.shape)
    break
        
vid.release()
cv.destroyAllWindows()

# Printing final results
print('\nTotal number of frames', f)
print('Total amount of time {:.5f} seconds'.format(t))
print()'''


Total number of frames 1
Total amount of time 0.10540 seconds



# Segmentation

In [3]:
def nothing(x):
    pass

In [4]:
# Fungsi trackbar untk proses masking
def masking(image):
    hsv = cv.cvtColor(image, cv.COLOR_BGR2HSV)
    #gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    min_r = cv.getTrackbarPos("min_red", "TrackBars")
    min_g = cv.getTrackbarPos("min_green", "TrackBars")
    min_b = cv.getTrackbarPos("min_blue", "TrackBars")
    
    lower = np.array([min_r, min_g, min_b])
    upper = np.array([255,255,255])
    mask = cv.inRange(hsv, lower, upper)
    
    #kernel = np.ones((5,5),np.uint8)
    #kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (11,11))
    #mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    
    #image_blurred = cv.GaussianBlur(gray, (0, 0), 3)
    #_,thresh = cv.threshold(image_blurred, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    
    return mask, min_r, min_g, min_b

In [7]:
path = r'C:\Users\Fernanda Januar\Downloads\fire4.mp4'
vid = cv.VideoCapture(path)
cv.namedWindow("TrackBars")

# Create TrackBar
## Minimum range
cv.createTrackbar("min_red", "TrackBars", 0, 255, nothing)
cv.createTrackbar("min_green", "TrackBars", 0, 255, nothing)
cv.createTrackbar("min_blue", "TrackBars", 0, 255, nothing)

f = 0
t = 0
while True:
    ret, frame = vid.read()
    if not ret:
        break
    
    # Masking + Segmentation
    mask, min_r, min_g, min_b = masking(frame)
    bit = cv.bitwise_and(frame, frame, mask=mask)
    
    hsv_rgb = cv.cvtColor(bit, cv.COLOR_HSV2RGB)
    gray = cv.cvtColor(hsv_rgb, cv.COLOR_RGB2GRAY)
    image_blurred = cv.GaussianBlur(gray, (0, 0), 3)
    _, thresh = cv.threshold(image_blurred, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    bit = cv.bitwise_and(frame, frame, mask=thresh)
    
    
    # Resize the Frame to 224,224,3 and convert to 1,224,224,3-----------
    img_resize = tf.keras.preprocessing.image.smart_resize(
        bit,
        (224, 224)
    )
    new_frame = np.expand_dims(img_resize, 0)
    
    # Predict the frames/images
    start = time.time()
    label_pred = np.argmax(model.predict(new_frame))
    
    end = time.time()
    f+=1
    t+=end-start
    #--------------------------------------------------------------------
    
    #Labels---------------------------------------------------------------------------------
    if label_pred==1:
        color = [0,0,255]
        lol = "FIRE"
    else:
        color = [255,0,0]
        lol = "Not_FIRE"
    
    text1 = "Predict :: {}".format(lol)
    text2 = "Frame number {0} took {1:.5f} seconds".format(f, end - start)
    text3 = "FPS: {}".format(round((f / t), 1))
    cv.putText(bit, text1, (30, 150), cv.FONT_HERSHEY_COMPLEX, 0.8, color, 2)
    cv.putText(bit, text2, (30, 190), cv.FONT_HERSHEY_COMPLEX, 0.8, (255,0,0), 2)
    cv.putText(bit, text3, (30, 230), cv.FONT_HERSHEY_COMPLEX, 0.8, (255,0,0), 2)
    #---------------------------------------------------------------------------------------'''
    
    cv.imshow("Frame_RGB", frame)
    cv.imshow("Frame_Mask", bit)
    if cv.waitKey(1) & 0xFF==ord("q"):
        break

    #break
vid.release()
cv.destroyAllWindows()


# Printing final results
print('\nTotal number of frames', f)
print('Total amount of time {:.5f} seconds'.format(t))

print("min red, min green, min blue: {0}, {1}, {2}".format(min_r, min_g, min_b))


Total number of frames 1097
Total amount of time 90.89765 seconds
min red, min green, min blue: 0, 68, 0


# Dont use this shit!!

In [46]:
cv.namedWindow("TrackBars")
cv.createTrackbar("min_red", "TrackBars", 0, 255, nothing)
cv.createTrackbar("min_green", "TrackBars", 0, 255, nothing)
cv.createTrackbar("min_blue", "TrackBars", 0, 255, nothing)

while True:
    mask, min_r, min_g, min_b = masking(frame)
    bitwise = cv.bitwise_and(frame, frame , mask=mask)
    
    #cv.imshow("Sample", frame)
    cv.imshow("Sample", mask)
    
    if cv.waitKey(1) & 0xFF==ord("q"):
        break

cv.destroyAllWindows()